# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl.utilities import get_primary_metrics
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails 
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice.aci import AciWebservice
from azureml.core.webservice import Webservice
import os
import joblib

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
if not os.path.exists('results'):
    os.makedirs('results')

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'predict-breast-cancer-automl'

# Check workspace details
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

experiment=Experiment(ws, experiment_name)

Workspace name: quick-starts-ws-143020
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-143020


In [4]:
# Check for existing cluster. Otherwise, create new cluster
try:
    cluster = ComputeTarget(workspace=ws, name="notebook143020")
    print("Cluster exists")
except:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V12', max_nodes=4)
    cluster = ComputeTarget.create(ws, "project-cluster", config)

cluster.wait_for_completion()

Cluster exists


In [5]:
train_data = Dataset.get_by_name(ws, name="breast_cancer_data")
train_data.take(5).to_pandas_dataframe()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=cluster,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="diagnosis",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             blocked_models=['XGBoostClassifier'],
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on notebook143020 with default configuration
Running on remote compute: notebook143020


Experiment,Id,Type,Status,Details Page,Docs Page
predict-breast-cancer-automl,AutoML_438291e4-ac86-4bf1-aa6e-d9ba21e642b3,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
        10   MinMaxScaler LogisticRegression                0:05:45       0.9190    0.9367
        12   StandardScalerWrapper GradientBoosting         0:05:40       0.8997    0.9367
        11   MaxAbsScaler RandomForest                      0:05:50       0.9138    0.9367
        13   StandardScalerWrapper LightGBM                 0:05:44       0.9279    0.9367
        14   MaxAbsScaler LightGBM                          0:06:23       0.9226    0.9367
        17   MinMaxScaler LightGBM                          0:05:45       0.9261    0.9367
        15   MaxAbsScaler LightGBM                          0:05:51       0.9120    0.9367
        16   RobustScaler LightGBM                          0:05:42       0.8980    0.9367
        18   RobustScaler KNN                               0:05:42       0.8962    0.9367
        19   MaxAbsScaler ExtremeRandomTrees                0:06:30       0.8822    0.9367
        20   Min

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [26]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [21]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
predict-breast-cancer-automl,AutoML_438291e4-ac86-4bf1-aa6e-d9ba21e642b3,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTIO

{'runId': 'AutoML_438291e4-ac86-4bf1-aa6e-d9ba21e642b3',
 'target': 'notebook143020',
 'status': 'Completed',
 'startTimeUtc': '2021-04-18T09:05:34.821992Z',
 'endTimeUtc': '2021-04-18T09:57:14.638131Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'notebook143020',
  'AMLSettingsJsonString': '{"path":null,"name":"predict-breast-cancer-automl","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group":"aml-quickstarts-143020","workspace_name":"quick-starts-ws-143020","region":"southcentralus","compute_target":"notebook143020","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [22]:
best_automl_run, best_automl_model = remote_run.get_output()
print(best_automl_run)

Run(Experiment: predict-breast-cancer-automl,
Id: AutoML_438291e4-ac86-4bf1-aa6e-d9ba21e642b3_26,
Type: azureml.scriptrun,
Status: Completed)


In [23]:
print(best_automl_run.properties['score'])

0.9401629072681704


In [24]:
best_automl_model._final_estimator

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('5',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=0.7922222222222222,
                                                                               importance_type='split',
                                                                               learning_rate=0.0842121052631579,
                                                     

In [25]:
#TODO: Save the best model
joblib.dump(best_automl_model, filename="results/automl_breast_cancer_model.joblib")

['results/automl_breast_cancer_model.joblib']

In [15]:
#Register best model
automl_model = remote_run.register_model(model_name=best_automl_run.properties['model_name'], 
                                           description='AutoML model')

print(automl_model.name, automl_model.id, automl_model.version, sep='\t')

AutoML438291e4a26	AutoML438291e4a26:1	1


In [18]:
#Another way to register model
description = 'automl_breast_cancer'
tags = {'area': 'health', 'type': 'classification'}
automl_model = best_automl_run.register_model(model_name='automodel',
                                             model_path='outputs/model.pkl',
                                             tags=tags,
                                             description=description)

print(automl_model.name, automl_model.id, automl_model.version, sep='\t')

automodel	automodel:1	1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service